### importing Library

In [0]:
#import argparse
import time
import csv
import pickle
import operator
import datetime
import os
import math
import numpy as np
import torch
from torch import nn
from torch.nn import Module, Parameter
import torch.nn.functional as F
import networkx as nx
import matplotlib.pyplot as plt

### Preprocesssing

In [0]:
def preprocessing(dataset="sample"):
    filename = 'sample_train-item-views.csv'
    if dataset == 'diginetica':
        filename = './dataset-train-diginetica/train-item-views.csv'
    elif dataset =='yoochoose':
        filename = './yoochoose-data/yoochoose-clicks.dat'
    print("-- Starting @ %ss" % datetime.datetime.now())    
    with open(filename, "r") as f:
        if dataset == 'yoochoose':
            reader = csv.DictReader(f, delimiter=',')
        else:
            reader = csv.DictReader(f, delimiter=';')
        sess_clicks = {}
        sess_date = {}
        ctr = 0
        curid = -1
        curdate = None
        for data in reader:
            sessid = data['session_id']
            if curdate and not curid == sessid:
                date = ''
                if dataset == 'yoochoose':
                    date = time.mktime(time.strptime(curdate[:19], '%Y-%m-%dT%H:%M:%S'))
                else:
                    date = time.mktime(time.strptime(curdate, '%Y-%m-%d'))
                sess_date[curid] = date
            curid = sessid
            if dataset == 'yoochoose':
                item = data['item_id']
            else:
                item = data['item_id'], int(data['timeframe'])
            curdate = ''
            if dataset == 'yoochoose':
                curdate = data['timestamp']
            else:
                curdate = data['eventdate']

            if sessid in sess_clicks:
                sess_clicks[sessid] += [item]
            else:
                sess_clicks[sessid] = [item]
            ctr += 1
        date = ''
        if dataset == 'yoochoose':
            date = time.mktime(time.strptime(curdate[:19], '%Y-%m-%dT%H:%M:%S'))
        else:
            date = time.mktime(time.strptime(curdate, '%Y-%m-%d'))
            for i in list(sess_clicks):
                sorted_clicks = sorted(sess_clicks[i], key=operator.itemgetter(1))
                sess_clicks[i] = [c[0] for c in sorted_clicks]
        sess_date[curid] = date
    print("-- Reading data @ %ss" % datetime.datetime.now())

    # Filter out length 1 sessions
    for s in list(sess_clicks):
        if len(sess_clicks[s]) == 1:
            del sess_clicks[s]
            del sess_date[s]

     # Count number of times each item appears
    iid_counts = {}
    for s in sess_clicks:
        seq = sess_clicks[s]
        for iid in seq:
            if iid in iid_counts:
                iid_counts[iid] += 1
            else:
                iid_counts[iid] = 1

    sorted_counts = sorted(iid_counts.items(), key=operator.itemgetter(1))

    length = len(sess_clicks)
    for s in list(sess_clicks):
        curseq = sess_clicks[s]
        filseq = list(filter(lambda i: iid_counts[i] >= 5, curseq))
        if len(filseq) < 2:
            del sess_clicks[s]
            del sess_date[s]
        else:
            sess_clicks[s] = filseq

    # Split out test set based on dates
    dates = list(sess_date.items())
    maxdate = dates[0][1]

    for _, date in dates:
        if maxdate < date:
            maxdate = date

    # 7 days for test
    splitdate = 0
    if dataset == 'yoochoose':
        splitdate = maxdate - 86400 * 1  # the number of seconds for a day：86400
    else:
        splitdate = maxdate - 86400 * 7

    print('Splitting date', splitdate)      # Yoochoose: ('Split date', 1411930799.0)
    tra_sess = filter(lambda x: x[1] < splitdate, dates)
    tes_sess = filter(lambda x: x[1] > splitdate, dates)

    # Sort sessions by date
    tra_sess = sorted(tra_sess, key=operator.itemgetter(1))     # [(session_id, timestamp), (), ]
    tes_sess = sorted(tes_sess, key=operator.itemgetter(1))     # [(session_id, timestamp), (), ]
    print(len(tra_sess))    # 186670    # 7966257
    print(len(tes_sess))    # 15979     # 15324
    print(tra_sess[:3])
    print(tes_sess[:3])
    print("-- Splitting train set and test set @ %ss" % datetime.datetime.now())

    # Choosing item count >=5 gives approximately the same number of items as reported in paper
    item_dict = {}
    # Convert training sessions to sequences and renumber items to start from 1
    def obtian_tra():
        train_ids = []
        train_seqs = []
        train_dates = []
        item_ctr = 1
        for s, date in tra_sess:
            seq = sess_clicks[s]
            outseq = []
            for i in seq:
                if i in item_dict:
                    outseq += [item_dict[i]]
                else:
                    outseq += [item_ctr]
                    item_dict[i] = item_ctr
                    item_ctr += 1
            if len(outseq) < 2:  # Doesn't occur
                continue
            train_ids += [s]
            train_dates += [date]
            train_seqs += [outseq]
        print(item_ctr)     # 43098, 37484
        return train_ids, train_dates, train_seqs


    # Convert test sessions to sequences, ignoring items that do not appear in training set
    def obtian_tes():
        test_ids = []
        test_seqs = []
        test_dates = []
        for s, date in tes_sess:
            seq = sess_clicks[s]
            outseq = []
            for i in seq:
                if i in item_dict:
                    outseq += [item_dict[i]]
            if len(outseq) < 2:
                continue
            test_ids += [s]
            test_dates += [date]
            test_seqs += [outseq]
        return test_ids, test_dates, test_seqs


    tra_ids, tra_dates, tra_seqs = obtian_tra()
    tes_ids, tes_dates, tes_seqs = obtian_tes()


    def process_seqs(iseqs, idates):
        out_seqs = []
        out_dates = []
        labs = []
        ids = []
        for id, seq, date in zip(range(len(iseqs)), iseqs, idates):
            for i in range(1, len(seq)):
                tar = seq[-i]
                labs += [tar]
                out_seqs += [seq[:-i]]
                out_dates += [date]
                ids += [id]
        return out_seqs, out_dates, labs, ids


    tr_seqs, tr_dates, tr_labs, tr_ids = process_seqs(tra_seqs, tra_dates)
    te_seqs, te_dates, te_labs, te_ids = process_seqs(tes_seqs, tes_dates)
    tra = (tr_seqs, tr_labs)
    tes = (te_seqs, te_labs)
    print(len(tr_seqs))
    print(len(te_seqs))
    print(tr_seqs[:3], tr_dates[:3], tr_labs[:3])
    print(te_seqs[:3], te_dates[:3], te_labs[:3])
    all = 0

    for seq in tra_seqs:
        all += len(seq)
    for seq in tes_seqs:
        all += len(seq)
    print('avg length: ', all/(len(tra_seqs) + len(tes_seqs) * 1.0))
    if dataset == 'diginetica':
        if not os.path.exists('diginetica'):
            os.makedirs('diginetica')
        pickle.dump(tra, open('diginetica/train.txt', 'wb'))
        pickle.dump(tes, open('diginetica/test.txt', 'wb'))
        pickle.dump(tra_seqs, open('diginetica/all_train_seq.txt', 'wb'))
    elif dataset == 'yoochoose':
        if not os.path.exists('yoochoose1_4'):
            os.makedirs('yoochoose1_4')
        if not os.path.exists('yoochoose1_64'):
            os.makedirs('yoochoose1_64')
        pickle.dump(tes, open('yoochoose1_4/test.txt', 'wb'))
        pickle.dump(tes, open('yoochoose1_64/test.txt', 'wb'))

        split4, split64 = int(len(tr_seqs) / 4), int(len(tr_seqs) / 64)
        print(len(tr_seqs[-split4:]))
        print(len(tr_seqs[-split64:]))

        tra4, tra64 = (tr_seqs[-split4:], tr_labs[-split4:]), (tr_seqs[-split64:], tr_labs[-split64:])
        seq4, seq64 = tra_seqs[tr_ids[-split4]:], tra_seqs[tr_ids[-split64]:]

        pickle.dump(tra4, open('yoochoose1_4/train.txt', 'wb'))
        pickle.dump(seq4, open('yoochoose1_4/all_train_seq.txt', 'wb'))

        pickle.dump(tra64, open('yoochoose1_64/train.txt', 'wb'))
        pickle.dump(seq64, open('yoochoose1_64/all_train_seq.txt', 'wb'))

    else:
        if not os.path.exists('sample'):
            os.makedirs('sample')
        pickle.dump(tra, open('sample/train.txt', 'wb'))
        pickle.dump(tes, open('sample/test.txt', 'wb'))
        pickle.dump(tra_seqs, open('sample/all_train_seq.txt', 'wb'))

    print('Done.')

In [0]:
class GNN(Module):
    def __init__(self, hidden_size, step=1):
        super(GNN, self).__init__()
        self.step = step
        self.hidden_size = hidden_size
        self.input_size = hidden_size * 2
        self.gate_size = 3 * hidden_size
        self.w_ih = Parameter(torch.Tensor(self.gate_size, self.input_size))
        self.w_hh = Parameter(torch.Tensor(self.gate_size, self.hidden_size))
        self.b_ih = Parameter(torch.Tensor(self.gate_size))
        self.b_hh = Parameter(torch.Tensor(self.gate_size))
        self.b_iah = Parameter(torch.Tensor(self.hidden_size))
        self.b_oah = Parameter(torch.Tensor(self.hidden_size))

        self.linear_edge_in = nn.Linear(self.hidden_size, self.hidden_size, bias=True)
        self.linear_edge_out = nn.Linear(self.hidden_size, self.hidden_size, bias=True)
        self.linear_edge_f = nn.Linear(self.hidden_size, self.hidden_size, bias=True)

    def GNNCell(self, A, hidden):
        input_in = torch.matmul(A[:, :, :A.shape[1]], self.linear_edge_in(hidden)) + self.b_iah
        input_out = torch.matmul(A[:, :, A.shape[1]: 2 * A.shape[1]], self.linear_edge_out(hidden)) + self.b_oah
        inputs = torch.cat([input_in, input_out], 2)
        gi = F.linear(inputs, self.w_ih, self.b_ih)
        gh = F.linear(hidden, self.w_hh, self.b_hh)
        i_r, i_i, i_n = gi.chunk(3, 2)
        h_r, h_i, h_n = gh.chunk(3, 2)
        resetgate = torch.sigmoid(i_r + h_r)
        inputgate = torch.sigmoid(i_i + h_i)
        newgate = torch.tanh(i_n + resetgate * h_n)
        hy = newgate + inputgate * (hidden - newgate)
        return hy

    def forward(self, A, hidden):
        for i in range(self.step):
            hidden = self.GNNCell(A, hidden)
        return hidden


class SessionGraph(Module):
    def __init__(self, param, n_node):
        super(SessionGraph, self).__init__()
        self.hidden_size = param["hiddenSize"]
        self.n_node = n_node
        self.batch_size = param["batchSize"]
        self.nonhybrid = param["onhybrid"]
        self.embedding = nn.Embedding(self.n_node, self.hidden_size)
        self.gnn = GNN(self.hidden_size, step=param["step"])
        self.linear_one = nn.Linear(self.hidden_size, self.hidden_size, bias=True)
        self.linear_two = nn.Linear(self.hidden_size, self.hidden_size, bias=True)
        self.linear_three = nn.Linear(self.hidden_size, 1, bias=False)
        self.linear_transform = nn.Linear(self.hidden_size * 2, self.hidden_size, bias=True)
        self.loss_function = nn.CrossEntropyLoss()
        self.optimizer = torch.optim.Adam(self.parameters(), lr=param["lr"], weight_decay=param["l2"])
        self.scheduler = torch.optim.lr_scheduler.StepLR(self.optimizer, step_size=param["lr_dc_step"], gamma=param["lr_dc"])
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1.0 / math.sqrt(self.hidden_size)
        for weight in self.parameters():
            weight.data.uniform_(-stdv, stdv)

    def compute_scores(self, hidden, mask):
        ht = hidden[torch.arange(mask.shape[0]).long(), torch.sum(mask, 1) - 1]  # batch_size x latent_size
        q1 = self.linear_one(ht).view(ht.shape[0], 1, ht.shape[1])  # batch_size x 1 x latent_size
        q2 = self.linear_two(hidden)  # batch_size x seq_length x latent_size
        alpha = self.linear_three(torch.sigmoid(q1 + q2))
        a = torch.sum(alpha * hidden * mask.view(mask.shape[0], -1, 1).float(), 1)
        if not self.nonhybrid:
            a = self.linear_transform(torch.cat([a, ht], 1))
        b = self.embedding.weight[1:]  # n_nodes x latent_size
        scores = torch.matmul(a, b.transpose(1, 0))
        return scores

    def forward(self, inputs, A):
        hidden = self.embedding(inputs)
        hidden = self.gnn(A, hidden)
        return hidden


def trans_to_cuda(variable):
    if torch.cuda.is_available():
        return variable.cuda()
    else:
        return variable


def trans_to_cpu(variable):
    if torch.cuda.is_available():
        return variable.cpu()
    else:
        return variable


def forward(model, i, data):
    alias_inputs, A, items, mask, targets = data.get_slice(i)
    alias_inputs = trans_to_cuda(torch.Tensor(alias_inputs).long())
    items = trans_to_cuda(torch.Tensor(items).long())
    A = trans_to_cuda(torch.Tensor(A).float())
    mask = trans_to_cuda(torch.Tensor(mask).long())
    hidden = model(items, A)
    get = lambda i: hidden[i][alias_inputs[i]]
    seq_hidden = torch.stack([get(i) for i in torch.arange(len(alias_inputs)).long()])
    return targets, model.compute_scores(seq_hidden, mask)


def train_test(model, train_data, test_data):
    model.scheduler.step()
    print('start training: ', datetime.datetime.now())
    model.train()
    total_loss = 0.0
    slices = train_data.generate_batch(model.batch_size)
    for i, j in zip(slices, np.arange(len(slices))):
        model.optimizer.zero_grad()
        targets, scores = forward(model, i, train_data)
        targets = trans_to_cuda(torch.Tensor(targets).long())
        loss = model.loss_function(scores, targets - 1)
        loss.backward()
        model.optimizer.step()
        total_loss += loss
        if j % int(len(slices) / 5 + 1) == 0:
            print('[%d/%d] Loss: %.4f' % (j, len(slices), loss.item()))
    print('\tLoss:\t%.3f' % total_loss)

    print('start predicting: ', datetime.datetime.now())
    model.eval()
    hit, mrr = [], []
    slices = test_data.generate_batch(model.batch_size)
    for i in slices:
        targets, scores = forward(model, i, test_data)
        sub_scores = scores.topk(20)[1]
        sub_scores = trans_to_cpu(sub_scores).detach().numpy()
        for score, target, mask in zip(sub_scores, targets, test_data.mask):
            hit.append(np.isin(target - 1, score))
            if len(np.where(score == target - 1)[0]) == 0:
                mrr.append(0)
            else:
                mrr.append(1 / (np.where(score == target - 1)[0][0] + 1))
    hit = np.mean(hit) * 100
    mrr = np.mean(mrr) * 100
    return hit, mrr

In [0]:
def build_graph(train_data):
    graph = nx.DiGraph()
    for seq in train_data:
        for i in range(len(seq) - 1):
            if graph.get_edge_data(seq[i], seq[i + 1]) is None:
                weight = 1
            else:
                weight = graph.get_edge_data(seq[i], seq[i + 1])['weight'] + 1
            graph.add_edge(seq[i], seq[i + 1], weight=weight)
    for node in graph.nodes:
        sum = 0
        for j, i in graph.in_edges(node):
            sum += graph.get_edge_data(j, i)['weight']
        if sum != 0:
            for j, i in graph.in_edges(i):
                graph.add_edge(j, i, weight=graph.get_edge_data(j, i)['weight'] / sum)
    return graph


def data_masks(all_usr_pois, item_tail):
    us_lens = [len(upois) for upois in all_usr_pois]
    len_max = max(us_lens)
    us_pois = [upois + item_tail * (len_max - le) for upois, le in zip(all_usr_pois, us_lens)]
    us_msks = [[1] * le + [0] * (len_max - le) for le in us_lens]
    return us_pois, us_msks, len_max


def split_validation(train_set, valid_portion):
    train_set_x, train_set_y = train_set
    n_samples = len(train_set_x)
    sidx = np.arange(n_samples, dtype='int32')
    np.random.shuffle(sidx)
    n_train = int(np.round(n_samples * (1. - valid_portion)))
    valid_set_x = [train_set_x[s] for s in sidx[n_train:]]
    valid_set_y = [train_set_y[s] for s in sidx[n_train:]]
    train_set_x = [train_set_x[s] for s in sidx[:n_train]]
    train_set_y = [train_set_y[s] for s in sidx[:n_train]]

    return (train_set_x, train_set_y), (valid_set_x, valid_set_y)


class Data():
    def __init__(self, data, shuffle=False, graph=None):
        inputs = data[0]
        inputs, mask, len_max = data_masks(inputs, [0])
        self.inputs = np.asarray(inputs)
        self.mask = np.asarray(mask)
        self.len_max = len_max
        self.targets = np.asarray(data[1])
        self.length = len(inputs)
        self.shuffle = shuffle
        self.graph = graph

    def generate_batch(self, batch_size):
        if self.shuffle:
            shuffled_arg = np.arange(self.length)
            np.random.shuffle(shuffled_arg)
            self.inputs = self.inputs[shuffled_arg]
            self.mask = self.mask[shuffled_arg]
            self.targets = self.targets[shuffled_arg]
        n_batch = int(self.length / batch_size)
        if self.length % batch_size != 0:
            n_batch += 1
        slices = np.split(np.arange(n_batch * batch_size), n_batch)
        slices[-1] = slices[-1][:(self.length - batch_size * (n_batch - 1))]
        return slices

    def get_slice(self, i):
        inputs, mask, targets = self.inputs[i], self.mask[i], self.targets[i]
        items, n_node, A, alias_inputs = [], [], [], []
        for u_input in inputs:
            n_node.append(len(np.unique(u_input)))
        max_n_node = np.max(n_node)
        for u_input in inputs:
            node = np.unique(u_input)
            items.append(node.tolist() + (max_n_node - len(node)) * [0])
            u_A = np.zeros((max_n_node, max_n_node))
            for i in np.arange(len(u_input) - 1):
                if u_input[i + 1] == 0:
                    break
                u = np.where(node == u_input[i])[0][0]
                v = np.where(node == u_input[i + 1])[0][0]
                u_A[u][v] = 1
            u_sum_in = np.sum(u_A, 0)
            u_sum_in[np.where(u_sum_in == 0)] = 1
            u_A_in = np.divide(u_A, u_sum_in)
            u_sum_out = np.sum(u_A, 1)
            u_sum_out[np.where(u_sum_out == 0)] = 1
            u_A_out = np.divide(u_A.transpose(), u_sum_out)
            u_A = np.concatenate([u_A_in, u_A_out]).transpose()
            A.append(u_A)
            alias_inputs.append([np.where(node == i)[0][0] for i in u_input])
        return alias_inputs, A, items, mask, targets

In [0]:
def SR_GNN(dataset='sample',batchSize=100 ,hiddenSize=100,epoch=30,lr=0.001,lr_dc=0.,lr_dc_step=3          
,l2=1e-5,step=1,patience=10,onhybrid=True,validation=True,valid_portion=0.1):

    param={}
    param["dataset"]=dataset
    param["batchSize"]=batchSize
    param["hiddenSize"]=hiddenSize
    param["epoch"]=epoch
    param["lr"]=lr
    param["lr_dc"]=lr_dc
    param["lr_dc_step"]=lr_dc_step
    param["l2"]=l2
    param["step"]=step
    param["patience"]=patience
    param["onhybrid"]=onhybrid
    param["validation"]=validation
    param["valid_portion"]=valid_portion
    train_data = pickle.load(open('./'+dataset + '/train.txt', 'rb'))
    if validation:
        train_data, valid_data = split_validation(train_data,valid_portion)
        test_data = valid_data
    else:
        test_data = pickle.load(open('../datasets/' + dataset + '/test.txt', 'rb'))
    # all_train_seq = pickle.load(open('../datasets/' + opt.dataset + '/all_train_seq.txt', 'rb'))
    # g = build_graph(all_train_seq)
    train_data = Data(train_data, shuffle=True)
    test_data = Data(test_data, shuffle=False)
    # del all_train_seq, g
    if dataset == 'diginetica':
        n_node = 43098
    elif dataset == 'yoochoose1_64' or dataset == 'yoochoose1_4':
        n_node = 37484
    else:
        n_node = 310

    model = trans_to_cuda(SessionGraph(param, n_node))

    start = time.time()
    best_result = [0, 0]
    best_epoch = [0, 0]
    bad_counter = 0
    for epoch in range(param["epoch"]):
        print('-------------------------------------------------------')
        print('epoch: ', epoch)
        hit, mrr = train_test(model, train_data, test_data)
        flag = 0
        if hit >= best_result[0]:
            best_result[0] = hit
            best_epoch[0] = epoch
            flag = 1
        if mrr >= best_result[1]:
            best_result[1] = mrr
            best_epoch[1] = epoch
            flag = 1
        print('Best Result:')
        print('\tRecall@20:\t%.4f\tMMR@20:\t%.4f\tEpoch:\t%d,\t%d'% (best_result[0], best_result[1], best_epoch[0], best_epoch[1]))
        bad_counter += 1 - flag
        if bad_counter >= param["patience"]:
            break
    print('-------------------------------------------------------')
    end = time.time()
    print("Run time: %f s" % (end - start))
    plt.plot


In [0]:
path = './code/SR-GNN-master/datasets'
 
files = os.listdir(path)
for name in files:
    print(name)

dataset-train-diginetica
yoochoose-data
preprocess.py
sample
tasty.csv
diginetica
sample_train-item-views.csv


In [0]:
os.chdir(path)
preprocessing()
SR_GNN()

-- Starting @ 2019-11-06 03:16:49.969296s
-- Reading data @ 2019-11-06 03:16:50.290404s
Splitting date 1464134400.0
469
47
[('2671', 1451952000.0), ('1211', 1452384000.0), ('3780', 1452384000.0)]
[('1864', 1464220800.0), ('1867', 1464220800.0), ('1868', 1464220800.0)]
-- Splitting train set and test set @ 2019-11-06 03:16:50.302245s
310
1205
99
[[1, 2], [1], [4]] [1451952000.0, 1451952000.0, 1452384000.0] [3, 2, 5]
[[282], [281, 308], [281]] [1464220800.0, 1464220800.0, 1464220800.0] [282, 281, 308]
avg length:  3.5669291338582676
Done.
-------------------------------------------------------
epoch:  0
start training:  2019-11-06 03:16:51.101327


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:100: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule.See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


[0/11] Loss: 5.7555
[3/11] Loss: 5.7213
[6/11] Loss: 5.6675
[9/11] Loss: 5.6276
	Loss:	62.578
start predicting:  2019-11-06 03:16:52.015838
Best Result:
	Recall@20:	52.0661	MMR@20:	32.1141	Epoch:	0,	0
-------------------------------------------------------
epoch:  1
start training:  2019-11-06 03:16:52.067061
[0/11] Loss: 5.5724
[3/11] Loss: 5.4866
[6/11] Loss: 5.4375
[9/11] Loss: 5.2832
	Loss:	59.661
start predicting:  2019-11-06 03:16:52.650035
Best Result:
	Recall@20:	52.0661	MMR@20:	32.1141	Epoch:	0,	0
-------------------------------------------------------
epoch:  2
start training:  2019-11-06 03:16:52.699100
[0/11] Loss: 5.1694
[3/11] Loss: 5.2855
[6/11] Loss: 5.3208
[9/11] Loss: 5.1034
	Loss:	56.630
start predicting:  2019-11-06 03:16:53.272009
Best Result:
	Recall@20:	52.0661	MMR@20:	32.1141	Epoch:	0,	0
-------------------------------------------------------
epoch:  3
start training:  2019-11-06 03:16:53.313284
[0/11] Loss: 5.1481
[3/11] Loss: 5.2510
[6/11] Loss: 4.9598
[9/11] 